In [1]:
%pylab inline
import pandas
import sqlite3
import pymc3
import seaborn as sns
from sklearn import *
from tqdm import tqdm
import ml_metrics

Populating the interactive namespace from numpy and matplotlib


/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
con = sqlite3.connect('/tmp/data.sqlite3')

In [3]:
batch_data = pandas.read_sql('''
    SELECT week_num, 
           sales_depo,
           sales_channel,
           route_id,
           client_id,
           product_id,
           adjusted_demand
      FROM data
     WHERE adjusted_demand is not null
           AND week_num < 8
           AND product_id < 1000
    ''', con=con)

In [4]:
n_products = batch_data.product_id.nunique()
n_depos = batch_data.sales_depo.nunique()
n_depos, n_products

(366, 47)

In [5]:
sales_depo_ids = {depo: id for id, depo in enumerate(batch_data.sales_depo.unique())}
product_id_ids = {product: id for id, product in enumerate(batch_data.product_id.unique())}

task_data = pandas.DataFrame([], index=batch_data.index)
task_data['sales_depo_id'] = batch_data.sales_depo.apply(sales_depo_ids.get)
task_data['product_id'] = batch_data.product_id.apply(product_id_ids.get)
task_data['adjusted_demand'] = batch_data.adjusted_demand

In [6]:
samps = task_data

In [63]:
simple_mlm = pymc3.Model()
with simple_mlm:
    product_id_rate = pymc3.Normal('product_id_rate', 100, shape=n_products)
    sales_depo_rate = pymc3.Normal('sales_depo_rate', 100, shape=n_depos)
    intercept = pymc3.Normal('intercept', 0, 100)
    #demand_variance = pymc3.HalfNormal('demand_variance', 10)
    
    sales_rate = pymc3.Deterministic(
        'sales_rate', 
        abs(product_id_rate[samps.product_id] + sales_depo_rate[samps.sales_depo_id] + intercept))
    
    adjusted_demand_est = pymc3.Poisson(
        'adjusted_demand', 
        sales_rate, #demand_variance,
        observed=samps.adjusted_demand)

In [64]:
with simple_mlm:
    trace = pymc3.sample(500, step=pymc3.NUTS())
    samples = pymc3.sample_ppc(trace[-100:])

 [-----------------100%-----------------] 501 of 500 complete in 58692.8 sec

In [65]:
ml_metrics.rmsle(
    abs(samps.adjusted_demand),
    abs(samples['adjusted_demand'].mean(axis=0)),
)

0.66688735758168283

In [ ]:
pymc3.traceplot(trace[-200:], ['sales_depo_rate'])

# Benchmarks

This compares various distributions for the adjusted demand estimate.

## Metropolis sampling 
- Normal: `1.1179161451846058`
- Negative binomial: `1.4359957577593117`
- Poisson: `1.4215856914577272`

## NUTS sampling

- Normal: `0.66091028618252567`
- Negative binomial: `1.4360295077622596`
- Poisson: `0.66688735758168283`